In [5]:
import pandas as pd
import numpy as np
import nltk
import pymongo
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from dask import bag

In [34]:
client = pymongo.MongoClient()
db = client.beerinvocate
db_reviews = db.reviews
review_df = pd.DataFrame(db_reviews.find()).drop(columns=['_id'])
#review_df = review_df.drop(columns=['polarity', 'subjectivity', 'review_pp1', 'review_pp2'])

## Experimental part of speech tagging

In [16]:
payload = rtext.str.split()
part = 64
payload = [
    payload[i:i+len(payload)//part] for i in range(0, len(payload), len(payload)//part)
]
b = bag.from_sequence(payload)
mapped = b.map(lambda x: nltk.pos_tag_sents(x))
sanitized = mapped.compute()

## Find phrases for the MWETokenizer

In [9]:
phrases = Phrases.load('../models/phrases.model')
def check_vocab(model, filter, score_threshold=300):
    ### Lookup a word and see if it's found in any common phrases.
    return [
        (str(phrase)[2:-1], score)
        for phrase, score in model.vocab.items()
        if str(phrase).find(filter) > -1 and score > score_threshold
    ]
faint_hinting = check_vocab(phrases, 'faint_')
light_hinting = check_vocab(phrases, 'light_')
print(*((tuple(x.split('_')), y) for x, y in check_vocab(phrases, 'pineapple')), sep='\n')

In [ ]:
model = Word2Vec.load('../models/w2v_reviews.model')
model.wv.most_similar(['weather'])

## Experimental Review Context Filter
Rationale:  Remove words that are irrelevant to each beer style.\
Possible Problems: Goal of the project is to recommend beers that are not typical of the style.  Could improve this by building a model for each beer, but many contexts will not be found per beer\
Known issues: Phrases often lose context within that beer style, and for the entire corpus

In [ ]:
#model = Word2Vec(review_df['review_pp1'].str.split(), size=100, window=5, min_count=5, workers=8, sg=1)
def get_flagged_words(model, review_pp1):
    flagged_words = []
    for word in set(word for review in review_pp1.str.split() for word in review):
        try:
            similar_words = model.wv.most_similar(word)
        except KeyError:
            flagged_words.append(word)
            continue
        if np.mean([v for k,v in similar_words]) < .80:
            flagged_words.append(word)
    return flagged_words

def generate_w2v_style_models(x):
    style, df = x
    model = Word2Vec(sentences=df.split_review_pp1, size=100, window=5, min_count=1, workers=4)
    model.save(f'w2v_style_models/{style}.model')
    flagged_words = set(get_flagged_words(model, df['review_pp1']))
    return [
        (i, ' '.join(word for word in review.split() if word not in flagged_words))
        for i, review in df[['review_pp1']].itertuples()
    ]

review_df['style'] = review_df['style'].str.replace('/', '')

b = bag.from_sequence(review_df.groupby('style'))
mapped = b.map(generate_w2v_style_models)
review_pp2 = mapped.compute()

review_df['review_pp2'] = np.nan
review_pp2 = [review_index for style_group in review_pp2 for review_index in style_group]
review_df = review_df.drop(columns=['review_pp2']).join(
    pd.DataFrame.from_records(review_pp2, columns=['index', 'review_pp2']).set_index('index')
)

In [164]:
review_df[review_df.review_pp2 != review_df.review_pp1].sample(25)[['style', 'review_pp1','review_pp2']]

,review_pp1,review_pp2
133210,l dark_red pour strong minimal potent raspberr...,l dark_red pour strong minimal potent raspberr...
13859,pale_ale delicious want pale_ale clear gold ev...,pale_ale delicious want pale_ale clear gold ev...
68915,haze pale orange fluffy white none spice coria...,haze pale orange fluffy white none spice coria...
53758,definitely live hype light_yellow straw mostly...,definitely light_yellow straw mostly pineapple...
152271,tap doherty’s house warwick ri tapered dark br...,tap doherty’s house warwick ri tapered brown l...
92333,gold pale amber producing huge point difficult...,gold pale amber producing huge point difficult...
153288,allaglass realize wa le hop black brown edge c...,realize wa le hop black brown edge cream reduc...
37011,canned december th according julian bottom bro...,canned th according julian bottom brought back...
113866,mug red brown clear vigorous pour yield mug ai...,mug brown clear vigorous pour yield mug airy s...
25325,mahogany brown light_tan reduces quickly biscu...,mahogany brown light_tan reduces biscuit malt ...


In [297]:
reviews.drop()
reviews_experimental.insert_many(review_df.to_dict(orient='records'))